<a href="https://colab.research.google.com/github/vis7/recommendation_system/blob/main/tensorflow_recommender_tutorial/movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installation
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 4.0 MB 5.4 MB/s 


In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

Below we are loading two diffrent part of the same dataset so we can get info with any of the part (ratings or movies)

In [22]:
# loading data
ratings, ratings_info = tfds.load('movielens/100k-ratings', split='train', with_info=True)
movies, movies_info = tfds.load('movielens/100k-movies', split='train', with_info=True)

In [23]:
# printing basic details about dataset
print('movies info')
print(movies_info)

movies info
tfds.core.DatasetInfo(
    name='movielens',
    full_name='movielens/100k-movies/0.1.0',
    description="""
    This dataset contains a set of movie ratings from the MovieLens website, a movie
    recommendation service. This dataset was collected and maintained by [GroupLens]
    (https://grouplens.org/), a research group at the University of Minnesota. There
    are 5 versions included: "25m", "latest-small", "100k", "1m", "20m". In all
    datasets, the movies data and ratings data are joined on "movieId". The 25m
    dataset, latest-small dataset, and 20m dataset contain only movie data and
    rating data. The 1m dataset and 100k dataset contain demographic data in
    addition to movie and rating data.
    
    - "25m": This is the latest stable version of the MovieLens dataset. It is
    recommended for research purposes.
    - "latest-small": This is a small subset of the latest version of the MovieLens
    dataset. It is changed and updated over time by GroupLens

In [24]:
# showing some examples from the dataset
# when passing info then it show categorical variable alogn with it's respective integer representation
tfds.as_dataframe(ratings.take(4), ratings_info)

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,7 (Drama),b'357',"b""One Flew Over the Cuckoo's Nest (1975)""",46.0,879024327,True,b'138',4 (doctor/health care),b'doctor',4.0,b'53211'
1,25.0,4 (Comedy)14 (Romance),b'709',b'Strictly Ballroom (1992)',32.0,875654590,True,b'92',5 (entertainment),b'entertainment',2.0,b'80525'
2,18.0,4 (Comedy),b'412',"b'Very Brady Sequel, A (1996)'",24.0,882075110,True,b'301',17 (student),b'student',4.0,b'55439'
3,50.0,5 (Crime)7 (Drama),b'56',b'Pulp Fiction (1994)',50.0,883326919,True,b'60',4 (doctor/health care),b'healthcare',4.0,b'06472'


In [25]:
tfds.as_dataframe(movies.take(4), movies_info)

,movie_genres,movie_id,movie_title
0,4 (Comedy),b'1681',b'You So Crazy (1994)'
1,4 (Comedy)7 (Drama),b'1457',b'Love Is All There Is (1996)'
2,1 (Adventure)3 (Children),b'500',b'Fly Away Home (1996)'
3,0 (Action),b'838',b'In the Line of Duty 2 (1987)'


In [26]:
# selecting basic feature
ratings = ratings.map(lambda x:{
    'movie_title': x['movie_title'],
    'user_id': x['user_id']
})

movies = movies.map(lambda x: x['movie_title'])

After selecting only necessory features

In [30]:
tfds.as_dataframe(ratings.take(4), ratings_info)

,movie_title,user_id
0,"b""One Flew Over the Cuckoo's Nest (1975)""",b'138'
1,b'Strictly Ballroom (1992)',b'92'
2,"b'Very Brady Sequel, A (1996)'",b'301'
3,b'Pulp Fiction (1994)',b'60'


In [31]:
tfds.as_dataframe(movies.take(4), movies_info)

,
0,b'You So Crazy (1994)'
1,b'Love Is All There Is (1996)'
2,b'Fly Away Home (1996)'
3,b'In the Line of Duty 2 (1987)'


In [29]:
# creating integer indices from string lookup
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x:x['user_id']))

movie_title_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_title_vocabulary.adapt(movies)

In [35]:
user_ids_vocabulary.get_vocabulary()[:5]

['[UNK]', '405', '655', '13', '450']

In [38]:
movie_title_vocabulary.get_vocabulary()[:5]

['[UNK]',
 "Ulee's Gold (1997)",
 'That Darn Cat! (1997)',
 'Substance of Fire, The (1996)',
 'Sliding Doors (1998)']

In [39]:
# using vocabulary and getting original records
test_movies = ['That Darn Cat! (1997)', 'Substance of Fire, The (1996)']
movie_title_vocabulary(test_movies) # you will get respective indexes

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([2, 3])>

In [42]:
# definging a model
class MovieLensModel(tfrs.Model):
  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval
  ):
    super().__init__()
    self.user_model = user_model
    self.movie_model = movie_model
    self.task = task
  
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model(features['user_id'])
    movie_embeddings = self.movie_model(features['movie_title'])
    return self.task(user_embeddings, movie_embeddings)

In [43]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])

movie_model = tf.keras.Sequential([
    movie_title_vocabulary,
    tf.keras.layers.Embedding(movie_title_vocabulary.vocab_size(), 64)
])

task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [44]:
# creating retrieval model
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

In [45]:
# train for 3 epochs
model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3
25/25 [==============================] - 17s 558ms/step - factorized_top_k/top_1_categorical_accuracy: 9.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0052 - factorized_top_k/top_50_categorical_accuracy: 0.0450 - factorized_top_k/top_100_categorical_accuracy: 0.1000 - loss: 33107.5488 - regularization_loss: 0.0000e+00 - total_loss: 33107.5488
Epoch 2/3
25/25 [==============================] - 14s 561ms/step - factorized_top_k/top_1_categorical_accuracy: 1.4000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0052 - factorized_top_k/top_10_categorical_accuracy: 0.0148 - factorized_top_k/top_50_categorical_accuracy: 0.1057 - factorized_top_k/top_100_categorical_accuracy: 0.2113 - loss: 31008.3727 - regularization_loss: 0.0000e+00 - total_loss: 31008.3727
Epoch 3/3
25/25 [==============================] - 15s 577ms/step - factorized_top_k/top_1_categorical_accuracy: 3.2000e-04 - factorized_top_k/top_5_categor

In [ ]:
# use brute-force search to set up retrival using the trained representations
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title)))
)

In [ ]:
# getting some recommendations
_, titles = index(np.array(["42"]))
print(f"top k recommendations for user 42: {titles[0,:3]}")

top k recommendations for user 42: [b'Rent-a-Kid (1995)' b'Just Cause (1995)' b'House Arrest (1996)']
